In [ ]:
# Assume a collection of playlists, P (which they refer to as the Case Base).

# User inputs to the algorithm:
# - seed song (s)
# - desired length (L)
# Hyperparameter (k)

import pandas as pd
import os, sys, time, random
import json
import numpy as np
import keras
import sqlite3

feature_list = ["track_name", "artist_name", "album_name", "duration_ms"]

data = json.load(open("data/mpd/data/mpd.slice.0-999.json"))

playlists = pd.DataFrame(data["playlists"])

k = 20
seed = 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak'

best_k_playlists = playlists["tracks"][:k:]
# For now, ignore what we mean by "best".
best_k_playlists = [[song["track_uri"] for song in playlist] for playlist in playlists["tracks"]]
best_k_playlists

best_k_playlists = [p[0] for p in containing_sorted] # list of k sorted playlist pids

2023-02-22 14:47:36.155862: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 14:47:36.572024: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-22 14:47:36.642205: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-22 14:47:36.642221: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

[['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
  'spotify:track:6I9VzXrHxO9rA9A5euc8Ak',
  'spotify:track:0WqIKmW4BTrj3eJFmnCKMv',
  'spotify:track:1AWQoqb9bSvzTjaLralEkT',
  'spotify:track:1lzr43nnXAijIGYnCT8M8H',
  'spotify:track:0XUfyU2QviPAs6bxSpXYG4',
  'spotify:track:68vgtRHr7iZHpzGpon6Jlo',
  'spotify:track:3BxWKCI06eQ5Od8TY2JBeA',
  'spotify:track:7H6ev70Weq6DdpZyyTmUXk',
  'spotify:track:2PpruBYCo4H7WOBJ7Q2EwM',
  'spotify:track:2gam98EZKrF9XuOkU13ApN',
  'spotify:track:4Y45aqo9QMa57rDsAJv40A',
  'spotify:track:1HwpWwa6bnqqRhK8agG4RS',
  'spotify:track:20ORwCJusz4KS2PbTPVNKo',
  'spotify:track:7k6IzwMGpxnRghE7YosnXT',
  'spotify:track:1Bv0Yl01xBDZD4OQP93fyl',
  'spotify:track:4omisSlTk6Dsq2iQD7MA07',
  'spotify:track:7xYnUQigPoIDAMPVK79NEq',
  'spotify:track:6d8A5sAx9TfdeseDvfWNHd',
  'spotify:track:4pmc2AxSEq6g7hPVlJCPyP',
  'spotify:track:215JYyyUnrJ98NK3KEwu6d',
  'spotify:track:0uqPG793dkDDN7sCUJJIVC',
  'spotify:track:19Js5ypV6JKn4DMExHQbGc',
  'spotify:track:1JURww012QnWAw0zZ

In [2]:

candidate_songs = list(set(np.concatenate(best_k_playlists))) # remove duplicates
current_playlist = [seed] # Britney Spears - Toxic
candidate_playlists = [current_playlist]
candidate_songs

['spotify:track:5y7M1Ed2Vj49Pkslcwmfc5',
 'spotify:track:5j9iuo3tMmQIfnEEQOOjxh',
 'spotify:track:1SwZVVH9BnXtLRLi2cbFhw',
 'spotify:track:178OI1A3qjROeFeh8lmNwW',
 'spotify:track:40IQooNkPRVtn4zlE3ZFpA',
 'spotify:track:2HWWsTZxnLSUdFZ4gjsP5N',
 'spotify:track:3kJudfRjZMItdFYVCCaSi6',
 'spotify:track:0Aqlt4N5h6rUtKnd09VXpr',
 'spotify:track:4k6oRr8qhWDcqDtcTGlIT5',
 'spotify:track:3RWzLyBUqoBSX4jTAHo4TP',
 'spotify:track:3XJJP46XhBhNWyVJ7DjZtO',
 'spotify:track:4MA2EibH9eGHgkbaTnJ3CR',
 'spotify:track:2yCf80rxfz3Dk7OCAnQvz1',
 'spotify:track:6zA2SEJJoiNByVMlQ4vgtH',
 'spotify:track:0bDl8ztrF7V5d4s7zjcCEi',
 'spotify:track:3UerZr7GF7qO2hQf6FwEbz',
 'spotify:track:2EZxM7Mw6haWQUtXmzsIDZ',
 'spotify:track:7brQHA2CgQpcMBiOlfiXYb',
 'spotify:track:4ecZErTEpadYyEuZfwqtzQ',
 'spotify:track:4wMfUdAxgiJno6flL6VW5r',
 'spotify:track:2MaCvfadvDHyrttQen5kv2',
 'spotify:track:2SHQl3Z4eNiYs4ihugT9lm',
 'spotify:track:2qgHh6CJMxYhwRRxaSwlmU',
 'spotify:track:6o8dOcU3e85aHPJJ9v7lFB',
 'spotify:track:

In [3]:
def match_all(pair):
    matches = set(pair).issubset
    return [playlist for playlist in best_k_playlists if matches(playlist)]

In [ ]:
def check_pair(x, y):
    N = 0
    containing_playlists = match_all([x, y])
    for playlist in containing_playlists:
        for i in range(len(playlist) - 1):
            if x == playlist[i]:
                if y == playlist[i+1]:
                    N += 1
                    break
            
    return 0

In [6]:
L = 8
while len(current_playlist) < L:
    t = current_playlist[0]
    T = current_playlist[-1]
    successors_of_current_playlist = []
    for u in candidate_songs:
        pre_occurences = check_pair(u, t)
        if pre_occurences > 0:
            successor_playlist = [u] + current_playlist
            successors_of_current_playlist.append((pre_occurences, successor_playlist))
        post_occurences = check_pair(T, u)
        if post_occurences > 0:
            successor_playlist = current_playlist + [u]
            successors_of_current_playlist.append((post_occurences, successor_playlist))
    if len(successors_of_current_playlist) == 0:
        # discard current_playlist
        print("failed to create playlist")
    else: # pop new current_playlist from candidate_playlists
    	# sort successors_of_current_playlist using some measure of quality 
        # For now, ignore the previous step
        # Then, later, we can implement the fancy sorting function
        successors_of_current_playlist = sorted(successors_of_current_playlist, key=lambda x: x[0])
        print(successors_of_current_playlist)
        current_playlist = successors_of_current_playlist[0]
        saved_playlists = successors_of_current_playlist[1::]
        # insert the saved_playlists into candidate_playlists
        candidate_playlists += saved_playlists
        # In fact, they should be inserted in such a way as to keep candidate_playlists 
        # ordered also by their quality, but ignore this for now

# If we reach here, we exited the loop, i.e. we have a playlist of length L
print(current_playlist)


failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to create playlist
failed to cr

KeyboardInterrupt: 